In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas
from io import StringIO
import sympy
import scipy
from sklearn.preprocessing import normalize
from pathlib import Path

In [2]:
# Funcion para correr .cpp

TP = "../src/main"
TEST_DIR = "../tests/"

def run_test(test_name: str, niter: int, eps: float): 
    !{TP} {TEST_DIR + test_name} {niter} {eps}

def read_result_autovalores(test_result: str):
    return pandas.read_csv(TEST_DIR + test_result + '_autovalores.out', names=["autovalores"])

def read_result_autovectores(test_result: str):
    return pandas.read_csv(TEST_DIR + test_result + '_autovectores.out' , names=["autovectores"])

### 1. Tests metodo de la potencia

In [3]:
# CASO SIMETRICO
A = np.array([
  [ 7,  2,  -3],
  [ 2,  2,  -2],
  [-3, -2,  -2]
])

# GUARDO TXT
np.savetxt("../tests/simetrico.txt", A, delimiter=' ')

# CORRO METODO
run_test("simetrico.txt", 100000, 1e-20)

# CARGO AUTOVALORES Y AUTOVECTORES
l = np.loadtxt("../tests/simetrico.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/simetrico.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l)
print(v)


print(A @ v[:, 0])
print(l[0] * v[:,0])

for i in range(len(A)):
    print(i)
    print(np.allclose(A @ v[:, i], l[i] * v[:,i], 1e-6))

Corriendo el programa...
[ 8.83835   -3.2849522  1.446602 ]
[[ 0.88455707 -0.22057717 -0.41097993]
 [ 0.34915668 -0.27109465  0.89699346]
 [-0.30927074 -0.9369383  -0.16278271]]
[ 7.81802505  3.08596897 -2.73344308]
[ 7.818025  3.085969 -2.733443]
0
True
1
True
2
True


In [6]:
# CASO (A) * (A traspuesta)
AT = A @ A.T
TA = A.T @ A

# GUARDO TXT
np.savetxt("../tests/AT.txt", AT, delimiter=' ')
np.savetxt("../tests/TA.txt", TA, delimiter=' ')

# CORRO METODO
run_test("AT.txt", 100000, 1e-20)
run_test("TA.txt", 100000, 1e-20)

# CARGO AUTOVALORES Y AUTOVECTORES
l_AT = np.loadtxt("../tests/AT.txt_autovalores.out", dtype='f', delimiter=' ')
v_AT = np.loadtxt("../tests/AT.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

l_TA = np.loadtxt("../tests/TA.txt_autovalores.out", dtype='f', delimiter=' ')
v_TA = np.loadtxt("../tests/TA.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l_AT)
print(l_TA)
print("Bingo :)" if np.allclose(l_AT,l_TA) else "Bongo :(")
print("Bingo :)" if np.allclose(v_AT,v_TA) else "Bongo :(")

Corriendo el programa...
Corriendo el programa...
[78.11643   10.79091    2.0926573]
[78.11643   10.79091    2.0926573]
Bingo :)
Bingo :)


### Ej 2 Juli

### PCA

In [3]:
paths = []
imgs = []
for path in sorted(list(Path('caras').rglob('*/*.pgm'))):
    paths.append(path)
    imgs.append(plt.imread(path)[::2,::2])
X = np.stack(imgs)

In [24]:
n = X.shape[0] # Cantidad de imagenes

X = X.reshape(n, -1) # X pasa a tener dimension n x (46 x 56)

X_centered = X - X.mean(0) # X.mean(0) devuelve un arreglo con la media de cada columna.

Mx = X_centered.T@X_centered/(n-1)  # Mx = Matriz de covarianza

# Calculo autovalores con numpy
# l, V = np.linalg.eigh(Mx)

# GUARDO TXT
np.savetxt("../tests/pca.txt", Mx, fmt='%.5f', delimiter=' ') # Trunca en 5 decimales (esta bien?)

# CORRO METODO
run_test("pca.txt", 10000, 1e-5)

# CARGO AUTOVALORES Y AUTOVECTORES
l = np.loadtxt("../tests/pca.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/pca.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l)


Corriendo el programa...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
^C


OSError: ../tests/pca.txt_autovalores.out not found.

### =======================================================================================

### 2DPCA

In [10]:
paths = []
imgs = []
for path in sorted(list(Path('caras').rglob('*/*.pgm'))):
    paths.append(path)
    imgs.append(plt.imread(path)[::2,::2])
X = np.stack(imgs)

La variante 2DPCA 2 considera una imagen en su espacio original como una matriz A ∈ Ra×b
y le asocia un feature vector Y ∈ Ra, mediante la transformacion lineal Y = AX siendo
X ∈ Rb el vector que maximiza la dispersion de los features vectors obtenidos para todo el
conjunto de imagenes. Se puede ver que la eleccion  ́optima para el vector de proyeccion X
se corresponde con el autovector de maximo autovalor de la denominada image covariance
matrix

In [5]:
print(f"Tengo {X.shape[0]} imagenes de tamano {X.shape[1]}x{X.shape[2]}")
# A es la imagen
imgs = X

n = imgs.shape[0] # Cantidad de imagenes

A_prom = np.zeros(shape=(imgs.shape[1], imgs.shape[2]))

for i in range (n):
    A_prom += imgs[i]

A_prom /= n


# G = image covariance matrix (quiero calcularle autovalores y autovectores)
sum = 0
for j in range (n): 
    sum += (np.transpose(imgs[j] - A_prom)) @ (imgs[j] - A_prom)
G = (1/n) * sum

# GUARDO TXT
np.savetxt("../tests/2dpca_g.txt", G, delimiter=' ')

# CORRO METODO
run_test("2dpca_g.txt", 100000, 1e-20)

# CARGO AUTOVALORES Y AUTOVECTORES
l = np.loadtxt("../tests/2dpca_g.txt_autovalores.out", dtype='f', delimiter=' ')
v = np.loadtxt("../tests/2dpca_g.txt_autovectores.out", dtype='f', delimiter=' ', usecols=range(3))

print(l)

# V = matriz de feature vectors
# U = base de autovectores ortonormales de G
# V = imgs[0] @ U


# Quiero computar los Zi = (AiX1, AiX2, ..., AiXk) de cada imagen
# donde cada AiXk representa el k-esimo feature vector de la imagen Ai



Tengo 410 imagenes de tamano 56x46
Corriendo el programa...
[1582449.2     599819.44    358704.34    233026.8     166467.75
  148195.08    123478.41     98200.13     79611.44     59310.723
   53157.484    45080.51     38945.5      35152.414    32514.654
   27217.822    25018.7      22815.846    21000.492    19401.13
   18807.984    18387.691    16664.152    16597.395    15355.004
   14703.68     13819.254    13014.369    12789.487    12005.537
   10399.625     9478.293     9208.34      8283.837     7242.492
    6673.762     6212.829     5999.4956    5536.206     5087.29
    4950.373     4628.532     4412.9263    4331.7974    4310.975
    4169.0947]


In [13]:
# Comparo con numpy y los autovalores dan bien
w, v = np.linalg.eig(G)
print(w)

[1582449.25555624  599819.43375503  358704.3313794   233026.79980762
  166467.75288011  148195.07387026  123478.40607429   98200.13125969
   79611.43491001   59310.72434661   53157.48267007   45080.51300064
   38945.49949404   35152.41305416   32514.6536101    27217.82148025
   25018.6990721    22815.84627941   21000.49217402   19401.13157525
   18807.98364156   18387.69198983   16664.15146487   16597.39502467
   15355.00405471   14703.68016929   13819.25434558   13014.36925274
   12789.48713691   12005.53705617   10399.62546503    9478.29255684
    9208.33973429    8283.83727704    7242.49239514    6673.76217712
    6212.82914369    5999.49582842    5536.20605205    5087.29013411
    4950.37282709    4628.53234203    4169.09476845    4412.9263435
    4310.97503377    4331.79732165]


### Ej 3